# CS 109A/STAT 121A/AC 209A/CSCI E-109A

## Standard Section 5: Partial Components Analysis (PCA) Fitting and Model Selection

**Harvard University**<br/>
**Fall 2017**<br/>
**Section Leaders: Albert Wu, Nathaniel Burbank<br/>**
**Instructors: Pavlos Protopapas, Kevin Rader, Rahul Dave, Margo Levine** 

<center>**Download this notebook from the CS109 repo**</center>

For this section, our goal is to review and further our understanding of the Partial Components Analysis (PCA) model. PCA is highly effective in applications to high dimensionional datasets, which we will use here. Specifically, this section is designed to help us answer Homework 4, part (h). 

Specifically, we will: 
    
    1. Review the basics of Partial Components Analysis and hone our intution
    2. Discuss implementation of PCA within Python and coding issues to keep in mind
    3. Use the principles of model selection we have learned in lecture to find a "best" PCA feature set.
    4. Compare our PCA model with other models we have fit in labs and lecture and discuss coefficient meanings.


For this section we will be using the following packages:

In [2]:
import sys
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 999)
pd.set_option('display.max_columns', 999)
pd.set_option('display.width', 1000)
pd.set_option('display.notebook_repr_html', True)
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
from statsmodels.api import OLS
from statsmodels.api import add_constant
from statsmodels.regression.linear_model import RegressionResults
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsRegressor
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
# Note --  Requires sklearn version .18 or higher  

from sklearn.metrics import r2_score
from collections import Counter
sns.set(style="ticks")
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

matplotlib.rcParams['figure.figsize'] = (13.0, 6.0)

assert(sys.version_info.major==3),print(sys.version)
# Python 3 or higher is required

/anaconda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


## Part (1): Load in our data and conduct basic EDA


We will first load in our dataset below and look at the first few rows. Then, we use the describe function to get a sense of the data.

In [14]:
crime_df = pd.read_csv('Crime.csv').drop(['Date', 'Year'], axis=1)
crime_df.head()

,Incidence,Temp,Dewpoint,Windspeed,Pressure,Precipitation,TMAX_C,TMIN_C,Month,DOW,Weekend,Season
0,182,26.444444,15.944444,8.5,1008.5,0.0,31.722222,21.722222,7,Sunday,1,Summer
1,295,24.333333,9.666667,7.9,1012.2,0.0,31.722222,20.000000,7,Monday,0,Summer
2,267,22.722222,9.666667,5.5,1016.0,0.0,28.888889,18.277778,7,Tuesday,0,Summer
3,250,23.166667,14.500000,6.7,1020.1,0.0,28.277778,18.277778,7,Wednesday,0,Summer
4,259,24.500000,15.611111,8.6,1021.3,0.0,30.000000,18.888889,7,Thursday,0,Summer


In [ ]:
crime_df.describe()

Convert the columns that are categorical variables into dummy variables by one-hot encoding.

In [5]:
categorical_columns = ['Month', 'Weekend', 'Season', 'DOW']
numerical_columns = ['Temp', 'Dewpoint', 'Windspeed', 'Pressure', 'Precipitation', 'TMAX_C', 'TMIN_C']
crime_df = pd.get_dummies(crime_df, columns=categorical_columns, drop_first=True)
crime_df.head()

,Incidence,Temp,Dewpoint,Windspeed,Pressure,Precipitation,TMAX_C,TMIN_C,Month_2,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12,Weekend_1,Season_Spring,Season_Summer,Season_Winter,DOW_Monday,DOW_Saturday,DOW_Sunday,DOW_Thursday,DOW_Tuesday,DOW_Wednesday
0,182,26.444444,15.944444,8.5,1008.5,0.0,31.722222,21.722222,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0
1,295,24.333333,9.666667,7.9,1012.2,0.0,31.722222,20.000000,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
2,267,22.722222,9.666667,5.5,1016.0,0.0,28.888889,18.277778,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
3,250,23.166667,14.500000,6.7,1020.1,0.0,28.277778,18.277778,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
4,259,24.500000,15.611111,8.6,1021.3,0.0,30.000000,18.888889,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0


Now, let's split this dataset up into a testing and training set.

In [6]:
train, test =  train_test_split(crime_df, test_size=.2, random_state=123)
train.shape,test.shape

((876, 29), (219, 29))

Now let us standarize the numerical variables only.

In [7]:
mean = train[numerical_columns].mean()
std = train[numerical_columns].std()

train[numerical_columns] = (train[numerical_columns] - mean)/std
test[numerical_columns] = (test[numerical_columns] - mean)/std

Now to let us subset and create convenient formats for parts in train and test

In [8]:
all_predictors = ['Temp','Dewpoint','Windspeed','Pressure','Precipitation','TMAX_C','TMIN_C','Month_2','Month_3','Month_4','Month_5','Month_6','Month_7','Month_8','Month_9','Month_10','Month_11','Month_12','Weekend_1','Season_Spring','Season_Summer','Season_Winter','DOW_Monday','DOW_Saturday','DOW_Sunday','DOW_Thursday','DOW_Tuesday','DOW_Wednesday']

X_train_df = train[all_predictors]
X_test_df  = test[all_predictors]
X_train_np = train[all_predictors].values
X_test_np  = test[all_predictors].values
y_train = train['Incidence'].values
y_test = test['Incidence'].values
y_test.shape

(219,)

## Part (2): Use subset selection to fit a linear regression model


Let's use the forward/backward subset selection method from HW 3 to fit a linear regression model on the data.

In [9]:
def step_forwards_backwards(df, y_val, direction='forward'):
    
    assert direction in ['forward', 'backward']
    
    y = y_val.reshape(-1,1)
    
    predictors = set(df.columns)
    selected_predictors = set() if direction=='forward' else set(predictors)
    
    n = df.shape[0]
    best_bic = np.inf
    
    best_bics = []
    best_models = []
    
    if direction == 'forward':
        X = np.ones(n).reshape(-1,1)
        X = np.concatenate([X, df[list(selected_predictors)].values], axis=1)
        while (True):
            
            possible_bic_scores = []
            possible_predictors = list(selected_predictors ^ predictors)
            
            if len(possible_predictors) == 0:
                break
                
            for predictor in possible_predictors:
                
                x_temp = np.concatenate([X, df[predictor].values.reshape(-1,1)], axis=1)
                model = OLS(endog=y, exog=x_temp).fit()
                bic = model.bic
                possible_bic_scores.append(bic)
                
            best_predictor_ix = np.argmin(possible_bic_scores)
            best_predictor = possible_predictors[best_predictor_ix]
            
            best_bic = np.min(possible_bic_scores)
            best_bics.append(best_bic)
            
            selected_predictors.add(best_predictor)            
            X = np.concatenate([X, df[best_predictor].values.reshape(-1,1)], axis=1)
            best_models.append(list(selected_predictors))

    else:

        while (True):
            possible_bic_scores = []
            possible_predictors = list(selected_predictors)

            if len(possible_predictors) == 0:
                break

            for predictor in possible_predictors:
                X = np.concatenate([np.ones(n).reshape(-1,1), df[list(selected_predictors - set([predictor]))].values], axis=1)
                model = OLS(endog=y, exog=X).fit()
                bic = model.bic
                possible_bic_scores.append(bic)

            best_predictor_ix = np.argmin(possible_bic_scores)
            best_predictor = possible_predictors[best_predictor_ix] 

            best_bic = possible_bic_scores[best_predictor_ix]
            selected_predictors.discard(best_predictor)
            
            best_bics.append(best_bic)
            best_models.append(list(selected_predictors))
            
    index_of_best_bic = np.argmin(best_bics)

    return best_models[index_of_best_bic]

Let's run the subset selection function and see which variables were included in the best model:

In [10]:
predictors_forward = # Your code goes here
predictors_forward

SyntaxError: invalid syntax (<ipython-input-10-3c4e73f96776>, line 1)

In [ ]:
predictors_backward = # Your code goes here
predictors_backward

Based on these variables, we can see what the R-squared values are for our training and testing sets.

In [11]:
X = sm.add_constant(X_train_df[predictors_backward])
X_test = sm.add_constant(X_test_df[predictors_backward])
y = train['Incidence'].values.reshape(-1,1)

model = OLS(endog=y, exog=X)
result = model.fit()

y_hat_train = result.predict()
y_hat_test = result.predict(exog=X_test)

print('Backward Selection Training R2 = ', r2_score(y_train, y_hat_train))
print('Backward Selection Testing R2 = ', r2_score(y_test, y_hat_test))

NameError: name 'predictors_backward' is not defined

In [ ]:
X = sm.add_constant(X_train_df[predictors_forward])
X_test = sm.add_constant(X_test_df[predictors_forward])
y = train['Incidence'].values.reshape(-1,1)

model = OLS(endog=y, exog=X)
result = model.fit()

y_hat_train = result.predict()
y_hat_test = result.predict(exog=X_test)

print('Forward Selection Training R2 = ', r2_score(y_train, y_hat_train))
print('Forward Selection Testing R2 = ', r2_score(y_test, y_hat_test))

## Part (3): Create a data frame with continuous predictors taken to polynomial power

Now, we will work with an example where we will manually take numeric predictors and take them to a polynomial power. The next step is a simple example to help you see how we can do this.

In [405]:
np.hstack((np.array([[1, 2,3], [4, 5,6],[7,8,9],[10,11,12]])**(i+1) for i in range(3)))

array([[   1,    2,    3,    1,    4,    9,    1,    8,   27],
       [   4,    5,    6,   16,   25,   36,   64,  125,  216],
       [   7,    8,    9,   49,   64,   81,  343,  512,  729],
       [  10,   11,   12,  100,  121,  144, 1000, 1331, 1728]])

We want to create a data frame with the continuous predictors taken up to a power 3, while keeping the rest of the categorical predictors the same. 

In [408]:
X_train_numerical_powers = # Your code goes here

print('Number of Total Predictors with Continuous Polynomial Terms Added is', X_train_num_powers.shape[1])

Number of Total Predictors with Continuous Polynomial Terms Added is 21


In [409]:
X_train_np_powers = np.concatenate((X_train_numerical_powers,X_train_df.drop(numerical_columns, axis=1)),axis=1)
X_train_df_powers = pd.DataFrame(X_train_np_powers)
newcolname = ['Temp', 'Dewpoint', 'Windspeed', 'Pressure', 'Precipitation', 'TMAX_C', 'TMIN_C', 'Temp^2', 'Dewpoint^2', 'Windspeed^2', 'Pressure^2', 'Precipitation^2', 'TMAX_C^2', 'TMIN_^2','Temp^3', 'Dewpoint^3', 'Windspeed^3', 'Pressure^3', 'Precipitation^3', 'TMAX_C^3', 'TMIN_C^3'] + list(X_train_df.drop(numerical_columns, axis=1))
X_train_df_powers.columns = newcolname
X_train_df_powers                                          
                             

,Temp,Dewpoint,Windspeed,Pressure,Precipitation,TMAX_C,TMIN_C,Temp^2,Dewpoint^2,Windspeed^2,Pressure^2,Precipitation^2,TMAX_C^2,TMIN_^2,Temp^3,Dewpoint^3,Windspeed^3,Pressure^3,Precipitation^3,TMAX_C^3,TMIN_C^3,Month_2,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12,Weekend_1,Season_Spring,Season_Summer,Season_Winter,DOW_Monday,DOW_Saturday,DOW_Sunday,DOW_Thursday,DOW_Tuesday,DOW_Wednesday
0,1.422812,1.580389,-0.574769,-0.350114,1.357830,1.233122,1.351858,2.024394,2.497629,0.330359,0.122580,1.843702,1.520589,1.827521,2.880331e+00,3.947224e+00,-1.898802e-01,-4.291693e-02,2.503433e+00,1.875072,2.470550e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,-1.517272,-1.799886,1.106358,0.018484,-0.345207,-1.212285,-1.515140,2.302115,3.239591,1.224028,0.000342,0.119168,1.469636,2.295649,-3.492934e+00,-5.830895e+00,1.354213e+00,6.315214e-06,-4.113767e-02,-1.781618,-3.478229e+00,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.472033,0.001912,-1.055091,-0.099994,-0.377340,-0.626240,-0.219640,0.222815,0.000004,1.113217,0.009999,0.142385,0.392177,0.048242,-1.051762e-01,6.989771e-09,-1.174545e+00,-9.998184e-04,-5.372774e-02,-0.245597,-1.059576e-02,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,-1.757621,-1.419225,1.556660,-0.350114,0.425979,-1.643828,-1.853096,3.089232,2.014199,2.423190,0.122580,0.181458,2.702170,3.433966,-5.429699e+00,-2.858600e+00,3.772083e+00,-4.291693e-02,7.729753e-02,-4.441903,-6.363470e+00,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,-0.108715,-0.130051,-0.094447,1.440219,-0.377340,0.226189,0.118317,0.011819,0.016913,0.008920,2.074232,0.142385,0.051162,0.013999,-1.284889e-03,-2.199571e-03,-8.424860e-04,2.987349e+00,-5.372774e-02,0.011572,1.656307e-03,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,-1.109238,-1.515659,2.006962,-1.482237,-0.120278,-1.270890,-0.839227,1.230408,2.297222,4.027895,2.197026,0.014467,1.615161,0.704301,-1.364815e+00,-3.481805e+00,8.083832e+00,-3.256512e+00,-1.740029e-03,-2.052692,-5.910685e-01,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
6,-0.617360,-0.688354,0.836177,0.887322,-0.088145,-0.567636,-0.501270,0.381134,0.473832,0.699192,0.787341,0.007770,0.322210,0.251272,-2.352970e-01,-3.261642e-01,5.846481e-01,6.986251e-01,-6.848471e-04,-0.182898,-1.259550e-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
7,-0.818583,-1.104544,0.896217,-0.402771,-0.377340,-0.732794,-1.058898,0.670078,1.220018,0.803205,0.162224,0.142385,0.536987,1.121266,-5.485144e-01,-1.347564e+00,7.198463e-01,-6.533928e-02,-5.372774e-02,-0.393501,-1.187307e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8,0.405520,0.905349,1.676740,-2.377403,3.028734,0.066359,0.625252,0.164447,0.819657,2.811458,5.652047,9.173230,0.004403,0.390940,6.668656e-02,7.420752e-01,4.714085e+00,-1.343720e+01,2.778327e+01,0.000292,2.444358e-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9,1.283074,1.184501,1.556660,0.439739,-0.377340,1.350331,1.188513,1.646279,1.403042,2.423190,0.193370,0.142385,1.823393,1.412563,2.112298e+00,1.661904e+00,3.772083e+00,8.503246e-02,-5.372774e-02,2.462185,1.678849e+00,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


We can do the same with the test set:

In [1]:
X_test_numerical_powers = # Your code goes here
X_test_np_powers = np.concatenate((X_test_numerical_powers,X_test_df.drop(numerical_columns, axis=1)),axis=1)
X_test_df_powers = pd.DataFrame(X_test_np_powers)
newcolname = ['Temp', 'Dewpoint', 'Windspeed', 'Pressure', 'Precipitation', 'TMAX_C', 'TMIN_C', 'Temp^2', 'Dewpoint^2', 'Windspeed^2', 'Pressure^2', 'Precipitation^2', 'TMAX_C^2', 'TMIN_^2','Temp^3', 'Dewpoint^3', 'Windspeed^3', 'Pressure^3', 'Precipitation^3', 'TMAX_C^3', 'TMIN_C^3'] + list(X_train_df.drop(numerical_columns, axis=1))
X_test_df_powers.columns = newcolname
X_test_df_powers.head()          

SyntaxError: invalid syntax (<ipython-input-1-1ef0c5fe47a5>, line 1)

We can do forward and backward selection as well on this new data frame. 

In [317]:
predictors_forward = step_forwards_backwards(X_train_df_powers, y_train, direction='forward')
predictors_forward

['DOW_Sunday',
 'DOW_Monday',
 'Windspeed',
 'Temp',
 'DOW_Wednesday',
 'Weekend_1',
 'Windspeed^3',
 'DOW_Thursday',
 'Season_Summer',
 'DOW_Tuesday',
 'Dewpoint^3']

In [319]:
predictors_backward = step_forwards_backwards(X_train_df_powers, y_train, direction='backward')
predictors_backward

['DOW_Wednesday',
 'Windspeed',
 'DOW_Thursday',
 'Dewpoint',
 'DOW_Sunday',
 'TMIN_C^3',
 'Season_Summer',
 'DOW_Monday',
 'DOW_Saturday',
 'DOW_Tuesday',
 'Temp']

In [320]:
X = sm.add_constant(X_train_df_powers[predictors_backward])
X_test = sm.add_constant(X_test_df_powers[predictors_backward])
y = train['Incidence'].values.reshape(-1,1)

model = OLS(endog=y, exog=X)
result = model.fit()

y_hat_train = result.predict()
y_hat_test = result.predict(exog=X_test)

print('Forward Selection Training R2 = ', r2_score(y_train, y_hat_train))
print('Forward Selection Testing R2 = ', r2_score(y_test, y_hat_test))

Forward Selection Training R2 =  0.422425372293
Forward Selection Testing R2 =  0.361986140851


In [ ]:
X = sm.add_constant(X_train_df_powers[predictors_forward])
X_test = sm.add_constant(X_test_df_powers[predictors_forward])
y = train['Incidence'].values.reshape(-1,1)

model = OLS(endog=y, exog=X)
result = model.fit()

y_hat_train = result.predict()
y_hat_test = result.predict(exog=X_test)

print('Forward Selection Training R2 = ', r2_score(y_train, y_hat_train))
print('Forward Selection Testing R2 = ', r2_score(y_test, y_hat_test))

## Part (4) Create the expanded matrix containing all terms

Let's now create a design matrix that includes all polynomial terms up to the third order, including all interactions. 

In [416]:
all_poly_terms = PolynomialFeatures(degree=3, interaction_only=False, include_bias=False)

X_train_full_poly = all_poly_terms.fit_transform(X_train_df)
X_test_full_poly = all_poly_terms.fit_transform(X_test_df)

print('number of total predictors', X_train_full_poly.shape[1])

number of total predictors 4494


In [417]:
X_train_full_poly.shape

(876, 4494)

We can use the following function to get an idea of the interactions between the variables.

In [418]:
all_poly_terms.get_feature_names()

['x0',
 'x1',
 'x2',
 'x3',
 'x4',
 'x5',
 'x6',
 'x7',
 'x8',
 'x9',
 'x10',
 'x11',
 'x12',
 'x13',
 'x14',
 'x15',
 'x16',
 'x17',
 'x18',
 'x19',
 'x20',
 'x21',
 'x22',
 'x23',
 'x24',
 'x25',
 'x26',
 'x27',
 'x0^2',
 'x0 x1',
 'x0 x2',
 'x0 x3',
 'x0 x4',
 'x0 x5',
 'x0 x6',
 'x0 x7',
 'x0 x8',
 'x0 x9',
 'x0 x10',
 'x0 x11',
 'x0 x12',
 'x0 x13',
 'x0 x14',
 'x0 x15',
 'x0 x16',
 'x0 x17',
 'x0 x18',
 'x0 x19',
 'x0 x20',
 'x0 x21',
 'x0 x22',
 'x0 x23',
 'x0 x24',
 'x0 x25',
 'x0 x26',
 'x0 x27',
 'x1^2',
 'x1 x2',
 'x1 x3',
 'x1 x4',
 'x1 x5',
 'x1 x6',
 'x1 x7',
 'x1 x8',
 'x1 x9',
 'x1 x10',
 'x1 x11',
 'x1 x12',
 'x1 x13',
 'x1 x14',
 'x1 x15',
 'x1 x16',
 'x1 x17',
 'x1 x18',
 'x1 x19',
 'x1 x20',
 'x1 x21',
 'x1 x22',
 'x1 x23',
 'x1 x24',
 'x1 x25',
 'x1 x26',
 'x1 x27',
 'x2^2',
 'x2 x3',
 'x2 x4',
 'x2 x5',
 'x2 x6',
 'x2 x7',
 'x2 x8',
 'x2 x9',
 'x2 x10',
 'x2 x11',
 'x2 x12',
 'x2 x13',
 'x2 x14',
 'x2 x15',
 'x2 x16',
 'x2 x17',
 'x2 x18',
 'x2 x19',
 'x2 x20',
 '

In [419]:
X_train_full_poly

array([[ 1.42281184,  1.58038879, -0.57476883, ...,  0.        ,
         0.        ,  0.        ],
       [-1.51727206, -1.79988626,  1.10635803, ...,  0.        ,
         0.        ,  0.        ],
       [-0.47203311,  0.001912  , -1.05509079, ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [-0.11430427, -0.2112585 ,  0.35585497, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.42228899,  0.05774237,  0.5359757 , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.41669948, -0.05391837, -1.02507067, ...,  0.        ,
         0.        ,  0.        ]])

If we wanted to drop the 0's, we can use the following function:

In [420]:
zero_column_index = np.where(~X_train_full_poly.any(axis=0))[0]

In [421]:
X_train_full_poly_nonzero_col = np.delete(X_train_full_poly, zero_column_index, axis = 1)
X_test_full_poly_nonzero_col = np.delete(X_test_full_poly, zero_column_index, axis = 1)

print(X_train_full_poly_nonzero_col)

[[ 1.42281184  1.58038879 -0.57476883 ...,  0.          0.          0.        ]
 [-1.51727206 -1.79988626  1.10635803 ...,  0.          0.          0.        ]
 [-0.47203311  0.001912   -1.05509079 ...,  0.          0.          0.        ]
 ..., 
 [-0.11430427 -0.2112585   0.35585497 ...,  0.          0.          0.        ]
 [ 0.42228899  0.05774237  0.5359757  ...,  0.          0.          0.        ]
 [ 0.41669948 -0.05391837 -1.02507067 ...,  0.          0.          0.        ]]


Now we can fit our PCA model:

In [447]:
pca = PCA(n_components=5)
pca.fit(X_train_full_poly)
train_pca = pca.transform(X_train_full_poly)
test_pca = pca.transform(X_test_full_poly)

print('Explained variance ratio:', pca.explained_variance_ratio_)


Explained variance ratio: [ 0.8129783   0.04373497  0.03401266  0.02028036  0.01556333]


We can obtain the corresponding coefficients of each principal component.

In [428]:
pca.components_
# First "row" of array corresponds to first component weights. 

array([[  2.89699969e-04,   8.16273796e-04,   2.21196880e-03, ...,
          0.00000000e+00,   0.00000000e+00,   1.63373238e-05],
       [ -2.10903042e-02,  -1.89535884e-02,   3.16788138e-02, ...,
         -0.00000000e+00,  -0.00000000e+00,   1.56530336e-04],
       [  6.60108855e-02,   6.63252686e-02,  -1.68813579e-02, ...,
         -0.00000000e+00,  -0.00000000e+00,  -1.08743378e-04],
       [ -1.22151524e-02,  -1.10596786e-02,  -3.84026203e-02, ...,
          0.00000000e+00,   0.00000000e+00,   2.21319814e-03],
       [  8.82561724e-03,   1.13001835e-02,   4.94705788e-02, ...,
          0.00000000e+00,   0.00000000e+00,   2.29230565e-03]])

If you recall, our weights squared had to sum to one. Let's see if this is the case:

In [429]:
# Your code goes here

array([ 1.,  1.,  1.,  1.,  1.])

Now we will look at each of the components, or weights from our 5 principal components. Because we centered our dataset, each value is essentially a correlation value. 

In [430]:
feature_frame = pd.DataFrame(pca.components_,columns=all_poly_terms.get_feature_names(),index = ['PC-1','PC-2','PC-3','PC-4','PC-5'])
feature_frame

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x0^2,x0 x1,x0 x2,x0 x3,x0 x4,x0 x5,x0 x6,x0 x7,x0 x8,x0 x9,x0 x10,x0 x11,x0 x12,x0 x13,x0 x14,x0 x15,x0 x16,x0 x17,x0 x18,x0 x19,x0 x20,x0 x21,x0 x22,x0 x23,x0 x24,x0 x25,x0 x26,x0 x27,x1^2,x1 x2,x1 x3,x1 x4,x1 x5,x1 x6,x1 x7,x1 x8,x1 x9,x1 x10,x1 x11,x1 x12,x1 x13,x1 x14,x1 x15,x1 x16,x1 x17,x1 x18,x1 x19,x1 x20,x1 x21,x1 x22,x1 x23,x1 x24,x1 x25,x1 x26,x1 x27,x2^2,x2 x3,x2 x4,x2 x5,x2 x6,x2 x7,x2 x8,x2 x9,x2 x10,x2 x11,x2 x12,x2 x13,x2 x14,x2 x15,x2 x16,x2 x17,x2 x18,x2 x19,x2 x20,x2 x21,x2 x22,x2 x23,x2 x24,x2 x25,x2 x26,x2 x27,x3^2,x3 x4,x3 x5,x3 x6,x3 x7,x3 x8,x3 x9,x3 x10,x3 x11,x3 x12,x3 x13,x3 x14,x3 x15,x3 x16,x3 x17,x3 x18,x3 x19,x3 x20,x3 x21,x3 x22,x3 x23,x3 x24,x3 x25,x3 x26,x3 x27,x4^2,x4 x5,x4 x6,x4 x7,x4 x8,x4 x9,x4 x10,x4 x11,x4 x12,x4 x13,x4 x14,x4 x15,x4 x16,x4 x17,x4 x18,x4 x19,x4 x20,x4 x21,x4 x22,x4 x23,x4 x24,x4 x25,x4 x26,x4 x27,x5^2,x5 x6,x5 x7,x5 x8,x5 x9,x5 x10,x5 x11,x5 x12,x5 x13,x5 x14,x5 x15,x5 x16,x5 x17,x5 x18,x5 x19,x5 x20,x5 x21,x5 x22,x5 x23,x5 x24,x5 x25,x5 x26,x5 x27,x6^2,x6 x7,x6 x8,x6 x9,x6 x10,x6 x11,x6 x12,x6 x13,x6 x14,x6 x15,x6 x16,x6 x17,x6 x18,x6 x19,x6 x20,x6 x21,x6 x22,x6 x23,x6 x24,x6 x25,x6 x26,x6 x27,x7^2,x7 x8,x7 x9,x7 x10,x7 x11,x7 x12,x7 x13,x7 x14,x7 x15,x7 x16,x7 x17,x7 x18,x7 x19,x7 x20,x7 x21,x7 x22,x7 x23,x7 x24,x7 x25,x7 x26,x7 x27,x8^2,x8 x9,x8 x10,x8 x11,x8 x12,x8 x13,x8 x14,x8 x15,x8 x16,x8 x17,x8 x18,x8 x19,x8 x20,x8 x21,x8 x22,x8 x23,x8 x24,x8 x25,x8 x26,x8 x27,x9^2,x9 x10,x9 x11,x9 x12,x9 x13,x9 x14,x9 x15,x9 x16,x9 x17,x9 x18,x9 x19,x9 x20,x9 x21,x9 x22,x9 x23,x9 x24,x9 x25,x9 x26,x9 x27,x10^2,x10 x11,x10 x12,x10 x13,x10 x14,x10 x15,x10 x16,x10 x17,x10 x18,x10 x19,x10 x20,x10 x21,x10 x22,x10 x23,x10 x24,x10 x25,x10 x26,x10 x27,x11^2,x11 x12,x11 x13,x11 x14,x11 x15,x11 x16,x11 x17,x11 x18,x11 x19,x11 x20,x11 x21,x11 x22,x11 x23,x11 x24,x11 x25,x11 x26,x11 x27,x12^2,x12 x13,x12 x14,x12 x15,x12 x16,x12 x17,x12 x18,x12 x19,x12 x20,x12 x21,x12 x22,x12 x23,x12 x24,x12 x25,x12 x26,x12 x27,x13^2,x13 x14,x13 x15,x13 x16,x13 x17,x13 x18,x13 x19,x13 x20,x13 x21,x13 x22,x13 x23,x13 x24,x13 x25,x13 x26,x13 x27,x14^2,x14 x15,x14 x16,x14 x17,x14 x18,x14 x19,x14 x20,x14 x21,x14 x22,x14 x23,x14 x24,x14 x25,x14 x26,x14 x27,x15^2,x15 x16,x15 x17,x15 x18,x15 x19,x15 x20,x15 x21,x15 x22,x15 x23,x15 x24,x15 x25,x15 x26,x15 x27,x16^2,x16 x17,x16 x18,x16 x19,x16 x20,x16 x21,x16 x22,x16 x23,x16 x24,x16 x25,x16 x26,x16 x27,x17^2,x17 x18,x17 x19,x17 x20,x17 x21,x17 x22,x17 x23,x17 x24,x17 x25,x17 x26,x17 x27,x18^2,x18 x19,x18 x20,x18 x21,x18 x22,x18 x23,x18 x24,x18 x25,x18 x26,x18 x27,x19^2,x19 x20,x19 x21,x19 x22,x19 x23,x19 x24,x19 x25,x19 x26,x19 x27,x20^2,x20 x21,x20 x22,x20 x23,x20 x24,x20 x25,x20 x26,x20 x27,x21^2,x21 x22,x21 x23,x21 x24,x21 x25,x21 x26,x21 x27,x22^2,x22 x23,x22 x24,x22 x25,x22 x26,x22 x27,x23^2,x23 x24,x23 x25,x23 x26,x23 x27,x24^2,x24 x25,x24 x26,x24 x27,x25^2,x25 x26,x25 x27,x26^2,x26 x27,x27^2,x0^3,x0^2 x1,x0^2 x2,x0^2 x3,x0^2 x4,x0^2 x5,x0^2 x6,x0^2 x7,x0^2 x8,x0^2 x9,x0^2 x10,x0^2 x11,x0^2 x12,x0^2 x13,x0^2 x14,x0^2 x15,x0^2 x16,x0^2 x17,x0^2 x18,x0^2 x19,x0^2 x20,x0^2 x21,x0^2 x22,x0^2 x23,x0^2 x24,x0^2 x25,x0^2 x26,x0^2 x27,x0 x1^2,x0 x1 x2,x0 x1 x3,x0 x1 x4,x0 x1 x5,x0 x1 x6,x0 x1 x7,x0 x1 x8,x0 x1 x9,x0 x1 x10,x0 x1 x11,x0 x1 x12,x0 x1 x13,x0 x1 x14,x0 x1 x15,x0 x1 x16,x0 x1 x17,x0 x1 x18,x0 x1 x19,x0 x1 x20,x0 x1 x21,x0 x1 x22,x0 x1 x23,x0 x1 x24,x0 x1 x25,x0 x1 x26,x0 x1 x27,x0 x2^2,x0 x2 x3,x0 x2 x4,x0 x2 x5,x0 x2 x6,x0 x2 x7,x0 x2 x8,x0 x2 x9,x0 x2 x10,x0 x2 x11,...,x14 x19 x20,x14 x19 x21,x14 x19 x22,x14 x19 x23,x14 x19 x24,x14 x19 x25,x14 x19 x26,x14 x19 x27,x14 x20^2,x14 x20 x21,x14 x20 x22,x14 x20 x23,x14 x20 x24,x14 x20 x25,x14 x20 x26,x14 x20 x27,x14 x21^2,x14 x21 x22,x14 x21 x23,x14 x21 x24,x14 x21 x25,x14 x21 x26,x14 x21 x27,x14 x22^2,x14 x22 x23,x14 x22 x24,x14 x22 x25,x14 x22 x26,x14 x22 x27,x14 x23^2,x14 x23 x24,x14 x23 x25,x14 x23 x26,x14

Now, let's take a look at the top correlation values for the first partial component. 

In [426]:
feature_frame[0:1].sort_values(by=['PC-1'], axis = 1, ascending = False).T[0:3]

,PC-1
x4^3,0.960024
x2 x4^2,0.146450
x4^2,0.101705


The first partial component really depends on x4^3, x2 x4^2 and x4^2. When each of these values increase, we would expect each of them to increase together.

Finally, let's look at the R-squared models fo the first 10 partial components. 

In [435]:

R2_pca_train = []
R2_pca_test = []

for i in range(10):
    pca = PCA(n_components=i+1)
    pca.fit(X_train_full_poly_nonzero_col)
    
    X_train_pca = pca.transform(X_train_full_poly_nonzero_col)
    X_test_pca = pca.transform(X_test_full_poly_nonzero_col)
    
    regression_model = LinearRegression(fit_intercept=True) 
    regression_model.fit(X_train_pca, y_train)
    
    R2_pca_train.append(regression_model.score(X_train_pca, y_train)) 
    R2_pca_test.append(regression_model.score(X_test_pca, y_test)) 
    
    print('Explained variance ratio for Model with',i+1, 'components:', pca.explained_variance_ratio_)
    

Explained variance ratio for Model with 1 components: [ 0.8129783]
Explained variance ratio for Model with 2 components: [ 0.8129783   0.04373497]
Explained variance ratio for Model with 3 components: [ 0.8129783   0.04373497  0.03401266]
Explained variance ratio for Model with 4 components: [ 0.8129783   0.04373497  0.03401266  0.02028036]
Explained variance ratio for Model with 5 components: [ 0.8129783   0.04373497  0.03401266  0.02028036  0.01556333]
Explained variance ratio for Model with 6 components: [ 0.8129783   0.04373497  0.03401266  0.02028036  0.01556333  0.00778059]
Explained variance ratio for Model with 7 components: [ 0.8129783   0.04373497  0.03401266  0.02028036  0.01556333  0.00778059
  0.00562893]
Explained variance ratio for Model with 8 components: [ 0.8129783   0.04373497  0.03401266  0.02028036  0.01556333  0.00778059
  0.00562893  0.00509171]
Explained variance ratio for Model with 9 components: [ 0.8129783   0.04373497  0.03401266  0.02028036  0.01556333  0.0

In [436]:
R2_pca_train

[0.0010658459075083559,
 0.03830594358760786,
 0.13381819106030979,
 0.13382136439024239,
 0.13585955685136197,
 0.13676941737898085,
 0.14451154996044147,
 0.14712808866011484,
 0.17240002211375349,
 0.17960666107411249]

In [437]:
R2_pca_test

[-0.0053200353011577661,
 0.051752057220783021,
 0.10653207172873704,
 0.10667438546230867,
 0.11083439698814734,
 0.11371485746532772,
 0.115030280594421,
 0.11572373943930712,
 0.13615410672470851,
 0.14113152029249787]